In [25]:
df = pandas.read_csv('svm-data.csv', header=None)
y = df[0]
X = df.loc[0:, 0:]

In [26]:
X = df.loc[:, 1:]
#[c нулевой строки : по конец строки, с столбца: до столбца]

In [1]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack

In [2]:
train = pandas.read_csv('salary-train.csv')

In [3]:
train

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
5,Sales and Marketing Assistant will provide adm...,Crawley,NaN,22500
6,Vacancy Ladieswear fashion Area Manager / Regi...,UK,permanent,32000
7,Reference: LR/JAN/**** Our client is one of th...,Bristol,permanent,30000
8,Sponsorship Manager London The Company A marke...,Central London,permanent,31500
9,"About Barclays Barclays moves, lends, invests ...",South East London,permanent,42499


In [4]:
# 2. Проведите предобработку:

In [5]:
def text_transform(text):
    # Приведите тексты к нижнему регистру (text.lower()).
    text = text.map(lambda t: t.lower())
    # Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой
    # замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться
    # методом replace у DataFrame, чтобы сразу преобразовать все тексты:
    text = text.replace('[^a-zA-Z0-9]', ' ', regex=True)
    return text

In [6]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова,
# которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [7]:
vec = TfidfVectorizer(min_df=5)
X_train_text = vec.fit_transform(text_transform(train['FullDescription']))

In [8]:
# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.

In [9]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

In [10]:
# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [11]:
enc = DictVectorizer()
X_train_cat = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))


In [12]:
# Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и
# категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией
# scipy.sparse.hstack.

In [13]:
X_train = hstack([X_train_text, X_train_cat])

In [14]:
X_train

<60000x24627 sparse matrix of type '<type 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [15]:
# 3. Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized.

In [16]:
y_train = train['SalaryNormalized']
model = Ridge(alpha=1)
model.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [17]:
# 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются
# ответом на задание. Укажите их через пробел.

In [19]:
test = pandas.read_csv('salary-test-mini.csv')
X_test_text = vec.transform(text_transform(test['FullDescription']))
X_test_cat = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test_text, X_test_cat])

y_test = model.predict(X_test)
print(1, '{:0.2f} {:0.2f}'.format(y_test[0], y_test[1]))

(1, '56528.66 37230.60')


## ПОнижение размерности и метод главных компонент

In [25]:
import pandas
from sklearn.decomposition import PCA
from numpy import corrcoef

In [20]:
# 1. Загрузите данные close_prices.csv. В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый
# день периода.

In [21]:
df = pandas.read_csv('close_prices.csv')
X = df.loc[:, 'AXP':]

In [28]:
df

,date,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002
5,2013-09-30,75.519997,117.500000,83.400002,23.430000,121.500000,58.560000,64.489998,23.889999,158.210007,...,28.730000,75.589996,33.820000,84.769997,71.610001,107.820000,191.100006,46.669998,73.959999,86.040001
6,2013-10-01,75.930000,117.750000,83.760002,23.240000,121.320000,58.689999,64.830002,24.170000,159.000000,...,28.889999,76.160004,34.060001,84.660004,72.580002,107.379997,193.220001,46.990002,73.589996,86.000000
7,2013-10-02,74.580002,117.839996,84.070000,23.320000,120.830002,58.989999,64.879997,24.330000,158.669998,...,29.010000,75.930000,33.939999,84.480003,72.570000,104.980003,191.820007,46.790001,73.720001,86.080002
8,2013-10-03,74.019997,115.239998,83.970001,23.010000,118.250000,57.759995,64.019997,24.100000,156.850006,...,28.770000,75.839996,33.639999,84.059998,72.519997,103.690002,188.649994,47.009998,73.160004,85.500000
9,2013-10-04,74.309998,117.199997,84.199997,23.020000,118.129997,58.649998,65.300003,24.049999,156.550003,...,29.000000,76.019997,33.750000,84.680000,72.989998,104.269997,190.479996,47.099998,72.800003,86.320000


In [27]:
X

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002
5,75.519997,117.500000,83.400002,23.430000,121.500000,58.560000,64.489998,23.889999,158.210007,75.849998,...,28.730000,75.589996,33.820000,84.769997,71.610001,107.820000,191.100006,46.669998,73.959999,86.040001
6,75.930000,117.750000,83.760002,23.240000,121.320000,58.689999,64.830002,24.170000,159.000000,76.260002,...,28.889999,76.160004,34.060001,84.660004,72.580002,107.379997,193.220001,46.990002,73.589996,86.000000
7,74.580002,117.839996,84.070000,23.320000,120.830002,58.989999,64.879997,24.330000,158.669998,76.320000,...,29.010000,75.930000,33.939999,84.480003,72.570000,104.980003,191.820007,46.790001,73.720001,86.080002
8,74.019997,115.239998,83.970001,23.010000,118.250000,57.759995,64.019997,24.100000,156.850006,75.879997,...,28.770000,75.839996,33.639999,84.059998,72.519997,103.690002,188.649994,47.009998,73.160004,85.500000
9,74.309998,117.199997,84.199997,23.020000,118.129997,58.649998,65.300003,24.049999,156.550003,75.779999,...,29.000000,76.019997,33.750000,84.680000,72.989998,104.269997,190.479996,47.099998,72.800003,86.320000


In [23]:
# 2. На загруженных данных обучите преобразование PCA с числом компоненты равным 10. Скольких компонент хватит,
# чтобы объяснить 90% дисперсии?

In [26]:
pca = PCA(n_components=10)
pca.fit(X.values)

var = 0
n_var = 0
for v in pca.explained_variance_ratio_:
    n_var += 1
    var += v
    if var >= 0.9:
        break

print(1, n_var)

(1, 4)


In [29]:
# 3. Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.

In [30]:
df_comp = pandas.DataFrame(pca.transform(X))
comp0 = df_comp[0]

In [31]:
comp0

0     -50.902404
1     -52.846909
2     -54.614439
3     -52.600566
4     -52.370123
5     -54.653412
6     -52.812575
7     -53.651146
8     -56.692727
9     -54.402655
10    -58.331645
11    -61.716265
12    -61.319400
13    -53.792524
14    -50.892634
15    -49.637408
16    -52.387615
17    -47.478543
18    -43.560335
19    -42.559280
20    -42.450271
21    -42.271248
22    -43.252728
23    -40.059345
24    -38.497263
25    -39.064923
26    -38.393044
27    -38.400558
28    -42.386170
29    -39.830574
         ...    
344    67.607802
345    74.206256
346    71.305000
347    68.041973
348    70.268799
349    71.435269
350    76.342232
351    74.859336
352    76.322836
353    75.621858
354    75.403574
355    80.946849
356    82.097270
357    83.045140
358    83.048730
359    84.251636
360    81.085307
361    88.953955
362    85.514365
363    83.441017
364    84.579747
365    77.973512
366    81.468090
367    72.450565
368    72.456036
369    81.290980
370    77.903534
371    83.7981

In [32]:
df_comp

,0,1,2,3,4,5,6,7,8,9
0,-50.902404,17.631677,7.736021,2.596010,2.377569,4.244055,12.883088,3.556570,-5.821974,-1.726040
1,-52.846909,19.146907,7.272546,4.618575,0.102182,3.935519,11.062930,3.432599,-5.633181,-2.012232
2,-54.614439,19.927358,8.109958,4.920347,-0.421454,4.555335,10.119224,4.175203,-5.388287,-2.752699
3,-52.600566,19.941571,6.948113,4.806243,0.319779,5.065505,9.849605,3.266579,-5.974574,-1.922746
4,-52.370123,22.289064,8.410868,3.833372,-2.464882,4.359897,7.988691,3.452709,-5.745204,-1.875131
5,-54.653412,24.190939,10.139937,3.492786,-4.591832,4.886501,6.940188,4.586919,-4.882303,-1.785527
6,-52.812575,23.005650,9.614647,2.953916,-3.605376,6.053104,7.250570,4.231228,-4.860103,-1.150869
7,-53.651146,23.884581,11.453830,3.563354,-4.270606,5.896830,6.119343,5.237782,-3.870597,-0.171450
8,-56.692727,25.622408,14.305737,3.039053,-7.108592,6.942510,5.332325,4.918475,-3.151208,-0.327393
9,-54.402655,24.754977,12.272721,3.548340,-6.778208,6.684524,4.653229,4.672059,-3.051907,0.247985


In [33]:
# 4. Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. Чему равна корреляция Пирсона между первой
# компонентой и индексом Доу-Джонса?

In [34]:
df2 = pandas.read_csv('djia_index.csv')
dji = df2['^DJI']
corr = corrcoef(comp0, dji)
print(2, corr[1, 0])

(2, 0.90965221930502371)


In [38]:
df2

,date,^DJI
0,2013-09-23,15401.379883
1,2013-09-24,15334.589844
2,2013-09-25,15273.259766
3,2013-09-26,15328.299805
4,2013-09-27,15258.240234
5,2013-09-30,15129.669922
6,2013-10-01,15191.700195
7,2013-10-02,15133.139648
8,2013-10-03,14996.480469
9,2013-10-04,15072.580078


In [35]:
# 5. Какая компания имеет наибольший вес в первой компоненте? Укажите ее название с большой буквы.

In [37]:
comp0_w = pandas.Series(pca.components_[0])
comp0_w_top = comp0_w.sort_values(ascending=False).head(1).index[0]
company = X.columns[comp0_w_top]
print(3, company)

(3, 'V')
